#### Intro

Let's build an matrix factorization autoencoder while taking the new [torch-sparse](https://github.com/rusty1s/pytorch_sparse) matrix multiplication library out for a spin.

This runs an experiment on MovieLens data which works reasonably well, and finds Star Trek movie vectors similar to other Star Trek movies -- promising enough to feel like this technique actually works well.

If this technique continues to work it opens up the path to making MF models that are more parallel and more scaleable. It also makes it easier to do variational / active learning stuff, and dovetails with adversarial methods.

#### The model

The big value of an autoencoder model is that we can predict a client vector in a single pass over just that client's ratings. This is great for real-time use cases and for scaling training: inferring a new client vector with a trained model is trivially parallelizeable. This is not the case at all in our current latent MF models, which need to see all the data to update client vectors.


0. **Encoder**. We encode ratings slightly differently than normal. Instead of saying +1 for a positive rating and -1 for a negative rating, we now categorically encode every combination of item and rating type. So if item=1 gets a 0.0 rating we use item_code=0, but if the same item=1 gets a different 1.0 rating we use a different item_code=1. MovieLens has ratings from 0.0 to 5.0 This means the *encoder* learns an embedding for (item_id=1, rating=0.0) and a new embedding for (item_id=1, rating=3.0).


1. **User Representation.** This creates a user vector from features (as in autoencoders) rather than memorizing one that recreates features (as in latent / PGM models). This user vector is constructed by matrix multiplying the sparse user-item ratings matrix $R_{ui}$ (of shape n_users x n_items) with a dense item vector matrix $Q_{ik}$ (of shape n_items x k dimensions). 

    $v = R \cdot Q / n$ 
    
    In practice we don't materialize the whole sparse $R$ matrix, but take chunks of rows. This follows the typical deep learning paradigm of minibatch training.
    
    We do the same to compute a user bias from rated items' biases. Here, $B$ is the item bias tensor of shape (n_items, 1)
    
    $b = R \cdot B / n$.

    The $n$ normalizes the user vectors and biases.  If you look carefully, $R \cdot Q$ and $R \cdot B$  are the *sum* of item vectors and biases for all items a single user rated. We want the average, so divide that user's rating count $n$.


2. **Sampling**. Mask the encoder ratings. By randomly masking half the ratings in any given iteration we're basically stochastically doing feature bagging and sampling. Because each iteration is random, over many epochs we'll still see all the data -- we're not downsampling. On the decoder, where we measure the likelihood of the data, we're not randomly sampling the data (maybe we should). I haven't tested whether this actually effective -- just a guess.


3. **Decoder.** The 'decoder' is in charge of using the user vector $v$ and the user bias $b$ to compute the likelihood. This procedes as a normal latent variable model. We add the user biases $b$ with decoder-specific item biases $c_i$ and interact the user vector $v$ generated in the encoder in the last step with decoder-specific item embedding $t_i$.

    $R_{i} \sim b + v \cdot t_i + c_i$
    
Note that here we learn a new decoder-specific item embedding $t_i$. This is completely distinct from the (item_id, rating id) embeddings we defined in the encoder. That means a single item has five encoder vectors (one for each of the five ratings) and has yet another representation in the decoder.

A few other differences:

- Ratings in previous autoencoders are shuffled. A single player may appear in the first batch and in the last batch of the same epoch. But in this model, we get user "blocks" of ratings -- a single user's ratings all fit into the same batch. That's because within each minibatch we want to compute a client vector from all of it's ratings at once.
- Validation as defined here isn't really what we want. The validation error is encoding a client's ratings into a compressed latent vector and then measuring the *reconstruction* loss. That's not quite what we want -- we want to give it half of a user's ratings and have it predict the reconstruction of the missing half, caring less about how well it recovers the given first half.

### Load preprocessed data

In [1]:
import numpy as np
fh = np.load('../data/dataset.npz')

# We have a bunch of feature columns and last column is the y-target
# Note pytorch is finicky about need int64 types
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

# We've already split into train & test
test_x = fh['test_x'].astype(np.int64)
test_y = fh['test_y']


n_user = int(fh['n_user'])
n_item = int(fh['n_item'])

Note that we'll change input item codes to be the outer product of (item_id) x (rating id), effectively getting five times as many item codes.

A hacky way to do this is to multiply the item id by 10 (e.g. shifting it over 1 digit), and then add the rating id.

In [2]:
item_code = train_x[:, 1] * 10 + train_y[:, 0].astype(int)
train_x = np.hstack((train_x, item_code[:, None]))

In [3]:
item_code = test_x[:, 1] * 10 + test_y[:, 0].astype(int)
test_x = np.hstack((test_x, item_code[:, None]))

In [4]:
# columns are user_id, item_id and other features 
# we won't use the 3rd and 4th columns
print(train_x)
print(' ')
print(train_y)

[[    1  1193    11    10 11935]
 [    1   914    26    10  9143]
 [    1  3408     7    10 34084]
 ...
 [ 6040   562    37     6  5625]
 [ 6040  1096   109     6 10964]
 [ 6040  1097    99     6 10974]]
 
[[5.]
 [3.]
 [4.]
 ...
 [5.]
 [4.]
 [4.]]


In [5]:
import torch
from torch import nn
import torch.nn.functional as F

In [6]:
def l2_regularize(array):
    loss = torch.sum(array ** 2.0)
    return loss

In [24]:
# this function multiplies a sparse matrix with a dense matrix
from torch_sparse import spmm
from torch.nn import Parameter


class MFAE(nn.Module):
    itr = 0
    frac = 0.5
    
    def __init__(self, n_encoder_item, n_decoder_item, k=18, c_vector=1.0, writer=None):
        super(MFAE, self).__init__()
        self.writer = writer
        self.k = k
        self.c_vector = c_vector
        self.n_item = n_item
        self.n_encoder_item = n_encoder_item
        self.encoder_bias = Parameter(torch.randn(n_encoder_item, 1) * 1e-6)
        self.encoder_vect = Parameter(torch.randn(n_encoder_item, k) * 1e-6)
        self.decoder_bias = Parameter(torch.randn(n_decoder_item, 1) * 1e-6)
        self.decoder_vect = Parameter(torch.randn(n_decoder_item, k) * 1e-6)
    
    def __call__(self, indices):
        # first column is user index, 2nd is item index and 3rd is
        # is an index over (item, rating_type), and not just item.
        # In the encoder, we use user index and item-rating-type index
        idx = torch.transpose(indices[:, [0, 4]], 1, 0)
        n_user_max = indices[:, 0].max() + 1

        # The encoder does a
        # matrix multiply between a 0 or 1 flag if a feature is present for a user
        # and the dense item representation matrix
        
        values = torch.ones(len(indices))
        count = 1 + torch.bincount(indices[:, 0], minlength=n_user_max).float()
        # this mask forces us to stochastically use half the
        # player's ratings to predict them all
        mask = torch.rand(len(indices)) > self.frac
        user_bias_sum = spmm(idx[:, mask], values[mask], n_user_max, self.encoder_bias)
        user_vect_sum = spmm(idx[:, mask], values[mask], n_user_max, self.encoder_vect)
        user_bias_mean = user_bias_sum / count[:, None]
        user_vect_mean = user_vect_sum / count[:, None]
        # Note user_vector is of size (max(user_idx in batch), k) 
        # and not (batchsize, k)!
    
        # Now we're in the decoder
        user_idx = indices[:, 0]
        item_idx = indices[:, 1]
        
        # Extract user/item bias/vectors
        # Note: we're using a different item representation in the decoder than the encoder
        user_bias = user_bias_mean[user_idx]
        user_vect = user_vect_mean[user_idx]
        item_bias = self.decoder_bias[item_idx]
        item_vect = self.decoder_vect[item_idx]

        # Compute likelihood
        user_item = (item_vect * user_vect).sum(dim=1)[:, None]
        log_odds = user_bias + item_bias + user_item
        return log_odds

    def loss(self, log_odds, target):
        loss_mse = F.mse_loss(log_odds, target.float())
        
        # Compute regularization
        prior_ie = l2_regularize(self.encoder_vect) * self.c_vector
        prior_id = l2_regularize(self.decoder_vect) * self.c_vector
        total = loss_mse + prior_ie + prior_id
        return total

In [25]:
import torch
from sklearn.utils import shuffle


class Loader():
    current = 0

    def __init__(self, x, y, batchsize=1024, do_shuffle=True):
        self.shuffle = shuffle
        self.x = x
        self.y = y
        self.batchsize = batchsize
        self.batches = range(0, len(self.y), batchsize)
        self.do_shuffle = do_shuffle
        if self.do_shuffle:
            # Every epoch re-shuffle the dataset
            self.x, self.y = shuffle(self.x, self.y)

    def __iter__(self):
        # Reset & return a new iterator
        if self.do_shuffle:
            self.x, self.y = shuffle(self.x, self.y, random_state=0)
        self.current = 0
        return self

    def __len__(self):
        # Return the number of batches
        return int(len(self.x) / self.batchsize)

    def __next__(self):
        n = self.batchsize
        if self.current + n >= len(self.y):
            raise StopIteration
        i = self.current
        xs = torch.from_numpy(self.x[i:i + n])
        ys = torch.from_numpy(self.y[i:i + n])
        self.current += n
        return (xs, ys)

### Train model

In [26]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss
from ignite.metrics import MeanSquaredError

from tensorboardX import SummaryWriter

from datetime import datetime

In [27]:
# Hyperparameters
lr = 1e-2
# Number of dimensions per user, item
k = 10
# regularization constant
c_vector = 1e-6

# Setup logging
log_dir = 'runs/simple_mf_01_' + str(datetime.now()).replace(' ', '_')
print(log_dir)
writer = SummaryWriter(log_dir=log_dir)

runs/simple_mf_01_2018-11-06_18:58:02.902128


In [28]:
train_y.dtype

dtype('float32')

In [29]:
n_item_enc = n_item * 10 + 1
n_item_dec = n_item + 1
model = MFAE(n_item_enc, n_item_dec, writer=writer, k=k, c_vector=c_vector)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [53]:
trainer = create_supervised_trainer(model, optimizer, model.loss)
metrics = {'accuracy': MeanSquaredError()}
evaluat = create_supervised_evaluator(model, metrics=metrics)

train_loader = Loader(train_x, train_y, batchsize=1024, do_shuffle=False)
test_loader = Loader(test_x, test_y, batchsize=1024, do_shuffle=False)

In [54]:
outs = []

def log_training_loss(engine, log_interval=1):
    global outs
    outs.append(engine.state.output)
    model.itr = engine.state.iteration
    if model.itr % log_interval == 0:
        fmt = "Epoch[{}] Iteration[{}/{}] Loss: {:.4f} Avg: {:.4f}"
        avg = np.mean(outs)
        msg = fmt.format(engine.state.epoch, engine.state.iteration, 
                         len(train_loader), engine.state.output, avg)
        print(msg)
        outs = []

trainer.add_event_handler(event_name=Events.EPOCH_COMPLETED, handler=log_training_loss)

def log_validation_results(engine):
    # When triggered, run the validation set
    model.frac = 0.0
    evaluat.run(test_loader)
    avg_accuracy = evaluat.state.metrics['accuracy']
    print("Epoch[{}] Validation MSE: {:.4f} ".format(engine.state.epoch, avg_accuracy))
    writer.add_scalar("validation/avg_accuracy", avg_accuracy, engine.state.epoch)
    model.frac = 0.5

trainer.add_event_handler(event_name=Events.EPOCH_COMPLETED, handler=log_validation_results)


model

MFAE()

#### Run model

In [55]:
trainer.run(train_loader, max_epochs=100)

Epoch[1] Iteration[879/879] Loss: 0.61 Avg: 0.6138
Epoch[1] Validation MSE: 0.8787 
Epoch[2] Iteration[1758/879] Loss: 0.61 Avg: 0.6100
Epoch[2] Validation MSE: 0.8822 
Epoch[3] Iteration[2637/879] Loss: 0.61 Avg: 0.6137
Epoch[3] Validation MSE: 0.8853 
Epoch[4] Iteration[3516/879] Loss: 0.61 Avg: 0.6092
Epoch[4] Validation MSE: 0.8807 
Epoch[5] Iteration[4395/879] Loss: 0.60 Avg: 0.6050
Epoch[5] Validation MSE: 0.8751 
Epoch[6] Iteration[5274/879] Loss: 0.60 Avg: 0.6038
Epoch[6] Validation MSE: 0.8825 
Epoch[7] Iteration[6153/879] Loss: 0.61 Avg: 0.6057
Epoch[7] Validation MSE: 0.8887 
Epoch[8] Iteration[7032/879] Loss: 0.61 Avg: 0.6123
Epoch[8] Validation MSE: 0.8752 
Epoch[9] Iteration[7911/879] Loss: 0.62 Avg: 0.6204
Epoch[9] Validation MSE: 0.8738 
Epoch[10] Iteration[8790/879] Loss: 0.60 Avg: 0.6042
Epoch[10] Validation MSE: 0.8739 
Epoch[11] Iteration[9669/879] Loss: 0.61 Avg: 0.6125
Epoch[11] Validation MSE: 0.8810 
Epoch[12] Iteration[10548/879] Loss: 0.61 Avg: 0.6107
Epoch[12

KeyboardInterrupt: 

### Introspect vectors

In [56]:

import pandas as pd
cols = ['item_id', 'title', 'tags']
df = pd.read_csv("../data/ml-1m/movies.dat", delimiter="::", engine="python", names=cols)

label_item = [str(iid) for iid in range(df.item_id.max() + 1)]
item_label = {}
for item, title in zip(df.item_id, df.title):
    label_item[item] = title
    item_label[title] = item
    
label_user = [str(uid) for uid in range(n_user)]

In [57]:
item_label['Star Trek: Generations (1994)']

329

In [58]:
lib = model.decoder_vect.data.numpy().copy()
#lib = lib / np.sqrt(((lib**2.0).sum(1)[:, None] + 1e-9))
vec = lib[329]
sim = ((lib - vec[None, :])**2.0).sum(1)
#sim = (lib * vec[None, :]).sum(1)

In [59]:
[label_item[l] for l in np.argsort(sim)[:10]]

['Star Trek: Generations (1994)',
 'Star Trek: Insurrection (1998)',
 'Star Trek III: The Search for Spock (1984)',
 'NeverEnding Story II: The Next Chapter, The (1990)',
 'Amistad (1997)',
 'Licence to Kill (1989)',
 'Star Trek V: The Final Frontier (1989)',
 'Star Trek: First Contact (1996)',
 'Mole People, The (1956)',
 'Song of the South (1946)']